In [33]:
import json
import joblib
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures, FunctionTransformer
import haversine as hs
from haversine import Unit


In [52]:
def regression_results(y_test, y_pred):
    explained_variance = metrics.explained_variance_score(y_test, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_test, y_pred) 
    mse = metrics.mean_squared_error(y_test, y_pred) 
    median_absolute_error = metrics.median_absolute_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    print('   explained_variance: ', round(explained_variance,4))    
    print('   r2: ', round(r2,4))
    print('   MAE: ', round(mean_absolute_error,4))
    print('   MSE: ', round(mse,4))
    print('   RMSE: ', round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)),4))

def save_model(model, name):
    rmse = str(int(np.sqrt(metrics.mean_squared_error(y_test, y_pred))))
    path = "../models/" + rmse + "_RMSE_" + str(name)
    joblib.dump(model, path)

In [3]:
df = pd.read_csv("../data/raw/lorapos-integration-data.csv", decimal=",")
data = []

for i in range(len(df)):
    hotspot = json.loads(df.hotspots[i])[0]
    d = [
        df.node_lng[i], 
        df.node_lat[i],
        hotspot["long"],
        hotspot["lat"],
        hotspot["snr"], 
        hotspot["rssi"], 
        hotspot["spreading"], 
        hotspot["frequency"], 
        hs.haversine((df.node_lat[i], df.node_lng[i]),(hotspot["lat"], hotspot["long"]),unit=hs.Unit.METERS)
    ]
    data.append(d)

df = pd.DataFrame(data, columns=["y_lng", "y_lat", "X_lng", "X_lat", "snr", "rssi", "spreading", "frequency", "distance"])
label = LabelEncoder()
df["spreading"] = label.fit_transform(df["spreading"])
df.head()

,y_lng,y_lat,X_lng,X_lat,snr,rssi,spreading,frequency,distance
0,12.053189,47.477710,12.166732,47.584275,6.0,-100.0,0,868.099976,14597.243233
1,12.053189,47.477714,12.166732,47.584275,5.5,-101.0,0,868.299988,14596.845865
2,12.053189,47.477714,12.166732,47.584275,3.8,-102.0,0,868.299988,14596.845865
3,12.000000,47.000000,12.176353,47.587865,-14.5,-121.0,0,868.099976,66706.964978
4,12.053189,47.477714,12.176353,47.587865,-14.8,-121.0,0,868.500000,15346.594383


In [4]:
y = df["distance"]
X = df.drop(["distance", "y_lng", "y_lat", "X_lng", "X_lat"], axis=1)
X

,snr,rssi,spreading,frequency
0,6.0,-100.0,0,868.099976
1,5.5,-101.0,0,868.299988
2,3.8,-102.0,0,868.299988
3,-14.5,-121.0,0,868.099976
4,-14.8,-121.0,0,868.500000
...,...,...,...,...
3619,-17.0,-133.0,0,868.500000
3620,-17.0,-133.0,0,868.500000
3621,-17.0,-133.0,0,868.500000
3622,-17.0,-133.0,0,868.500000


In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1007486)

### Linear Regression Model

In [54]:
lin = LinearRegression()
lin.fit(X_train, y_train)
y_pred = lin.predict(X_test)
save_model(lin, name="LinearRegression")
regression_results(y_test, y_pred)

   explained_variance:  0.287
   r2:  0.2855
   MAE:  403.9021
   MSE:  9649843.0535
   RMSE:  3106.4197


In [58]:
poly_transformer = PolynomialFeatures(degree=2)
X_poly_train = poly_transformer.fit_transform(X_train)
X_poly_test = poly_transformer.fit_transform(X_test)
linear_regressor = LinearRegression()
linear_regressor.fit(X_poly_train, y_train)
y_pred = linear_regressor.predict(X_poly_test)
save_model(linear_regressor, name="LinearRegression_PolynomialFeatures")
regression_results(y_test, y_pred)

   explained_variance:  0.433
   r2:  0.4302
   MAE:  347.5442
   MSE:  7694820.2517
   RMSE:  2773.9539


In [59]:
funktion_transformer = FunctionTransformer(np.log1p, validate=True)

X_log_train = X_train.copy()
X_log_train["rssi"] = X_log_train["rssi"]*-1
X_log_train["snr"] = X_log_train["snr"]**2

X_log_test = X_test.copy()
X_log_test["rssi"] = X_log_test["rssi"]*-1
X_log_test["snr"] = X_log_test["snr"]**2

X_log_train = funktion_transformer.transform(X_log_train)
X_log_test = funktion_transformer.transform(X_log_test)

linear_regressor = LinearRegression()
linear_regressor.fit(X_log_train, y_train)
y_pred = linear_regressor.predict(X_log_test)

save_model(linear_regressor, name="LinearRegression_FunctionTransformer")
regression_results(y_test, y_pred)

   explained_variance:  0.3301
   r2:  0.3284
   MAE:  384.8964
   MSE:  9070742.729
   RMSE:  3011.7674


f:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(
f:\Python\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but FunctionTransformer was fitted without feature names
  warnings.warn(


### Random Forest Regression

In [60]:
random_forest_regression = RandomForestRegressor(n_estimators=100, max_depth=20)
random_forest_regression.fit(X_train, y_train)
y_pred = random_forest_regression.predict(X_test)

save_model(random_forest_regression, name="RandomForestRegressor")
regression_results(y_test, y_pred)

   explained_variance:  0.419
   r2:  0.4168
   MAE:  256.9121
   MSE:  7875813.4604
   RMSE:  2806.388


### Randomized Search

In [61]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,500,10),
    "estimator__criterion": ["squared_error", "absolute_error", "poisson"],
    "estimator__min_samples_split":np.arange(1, 50),
    "estimator__min_samples_leaf":np.arange(1, 50),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__max_depth": np.arange(2, 20)
}


opt_rf = RandomizedSearchCV(rf, rgrid, cv = 5, scoring="neg_root_mean_squared_error", random_state=1007486)
opt_rf.fit(X_train, y_train)
y_pred = opt_rf.predict(X_test)
save_model(opt_rf, name="RandomForestRegressor_RandomizedSearchCV")
regression_results(y_test, y_pred)
print(opt_rf.best_params_)

   explained_variance:  0.3478
   r2:  0.3459
   MAE:  290.767
   MSE:  8833135.6948
   RMSE:  2972.0592
{'estimator__n_estimators': 150, 'estimator__min_samples_split': 34, 'estimator__min_samples_leaf': 7, 'estimator__max_features': 'sqrt', 'estimator__max_depth': 4, 'estimator__criterion': 'squared_error'}


### Gridsearch

In [62]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,200,10),
    "estimator__max_depth": np.arange(2, 20)
}


opt_rf = GridSearchCV(rf, rgrid, cv = 5, scoring="neg_root_mean_squared_error")
opt_rf.fit(X_train, y_train)
y_pred = opt_rf.predict(X_test)
save_model(opt_rf, name="RandomForestRegressor_GridSearchCV")
regression_results(y_test, y_pred)
print(opt_rf.best_params_)

   explained_variance:  0.416
   r2:  0.4139
   MAE:  257.4149
   MSE:  7915651.3475
   RMSE:  2813.4767
{'estimator__max_depth': 7, 'estimator__n_estimators': 80}


### GradientBoostingRegressor

In [63]:

pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("estimator", GradientBoostingRegressor())
    ]
)

grid = {
    "estimator__n_estimators": np.arange(10,500,10),
    "estimator__learning_rate": np.linspace(1e-3, 0.5,50),
    "estimator__max_depth": np.arange(1,15), 
    "estimator__min_samples_split": np.linspace(1e-7, 0.99,100),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__loss": ["squared_error", "absolute_error", "huber", "quantile"]
}

gbt_grid = RandomizedSearchCV(pipeline, grid, cv=5,scoring="neg_root_mean_squared_error", random_state=1007486)

gbt_grid.fit(X_train, y_train)
y_pred = gbt_grid.predict(X_test)
save_model(gbt_grid, name="GradientBoostingRegressor")
regression_results(y_test, y_pred)
print(gbt_grid.best_params_)

   explained_variance:  0.4466
   r2:  0.4449
   MAE:  244.7348
   MSE:  7496472.5269
   RMSE:  2737.9687
{'estimator__n_estimators': 290, 'estimator__min_samples_split': 0.7900000202020202, 'estimator__max_features': 'sqrt', 'estimator__max_depth': 7, 'estimator__loss': 'squared_error', 'estimator__learning_rate': 0.08246938775510204}


### Neural Network

In [67]:
pipe_MLPRegressor = Pipeline(
    [
        ('scaler',  StandardScaler()),
        ('MLPRegressor', MLPRegressor(random_state = 42, max_iter=1000))
    ]
)

grid_params_MLPRegressor = [{
    'MLPRegressor__solver': ['lbfgs'],
    'MLPRegressor__activation' : ['relu','logistic','tanh'],
    'MLPRegressor__hidden_layer_sizes':[(2,), (4,),(2,2),(4,4),(4,2),(10,10),(2,
2,2)],}]

CV_mlpregressor = GridSearchCV(estimator = pipe_MLPRegressor, param_grid = grid_params_MLPRegressor, cv = 5, return_train_score=True, verbose=0)

CV_mlpregressor.fit(X_train, y_train)
y_pred = CV_mlpregressor.predict(X_test)
save_model(gbt_grid, name="NeuralNetwork")
regression_results(y_test, y_pred)
print(CV_mlpregressor.best_params_)

f:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
f:\Python\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


   explained_variance:  0.4595
   r2:  0.4576
   MAE:  268.0448
   MSE:  7325858.0788
   RMSE:  2706.6322
{'MLPRegressor__activation': 'relu', 'MLPRegressor__hidden_layer_sizes': (4, 2), 'MLPRegressor__solver': 'lbfgs'}


### PCA

In [65]:
rf  = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("decomposer", PCA()),
        ("estimator", RandomForestRegressor(warm_start=True, oob_score=True))
    ]
)

rgrid = {
    "estimator__n_estimators": np.arange(50,500,10),
    "estimator__criterion": ["squared_error", "absolute_error", "poisson"],
    "estimator__min_samples_split":np.arange(1, 50),
    "estimator__min_samples_leaf":np.arange(1, 50),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__max_depth": np.arange(2, 20), 
    "decomposer__n_components": np.arange(1, 6)
}


opt_rf = RandomizedSearchCV(rf, rgrid, cv = 5, scoring="neg_root_mean_squared_error", random_state=1007486)
opt_rf.fit(X_train, y_train)
y_pred = opt_rf.predict(X_test)
save_model(opt_rf, name="RandomForestRegressor_PCA_RandomizedSearchCV")
regression_results(y_test, y_pred)
print(opt_rf.best_params_)

f:\Python\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "f:\Python\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "f:\Python\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "f:\Python\lib\site-packages\sklearn\ensemble\_forest.py", line 476, in fit
    trees = Parallel(
  File "f:\Python\lib\site-packages\joblib\parallel.py", line 1085, in __call__
    if self.dispatch_

   explained_variance:  0.4372
   r2:  0.4357
   MAE:  259.9988
   MSE:  7620983.9725
   RMSE:  2760.613
{'estimator__n_estimators': 360, 'estimator__min_samples_split': 8, 'estimator__min_samples_leaf': 19, 'estimator__max_features': 'sqrt', 'estimator__max_depth': 11, 'estimator__criterion': 'absolute_error', 'decomposer__n_components': 3}


In [66]:
pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler()),
        ("decomposer", PCA()),
        ("estimator", GradientBoostingRegressor())
    ]
)

grid = {
    "estimator__n_estimators": np.arange(10,500,10),
    "estimator__learning_rate": np.linspace(1e-3, 0.5,50),
    "estimator__max_depth": np.arange(1,15), 
    "estimator__min_samples_split": np.linspace(1e-7, 0.99,100),
    "estimator__max_features": ["sqrt", "log2"],
    "estimator__loss": ["squared_error", "absolute_error", "huber", "quantile"],
    "decomposer__n_components": np.arange(1, 7)
}

gbt_grid = RandomizedSearchCV(pipeline, grid, cv=5,scoring="neg_root_mean_squared_error", random_state=1007486)

gbt_grid.fit(X_train, y_train)
y_pred = gbt_grid.predict(X_test)
save_model(gbt_grid, name="GradientBoostingRegressor_PCA_RandomizedSearchCV")
regression_results(y_test, y_pred)
print(gbt_grid.best_params_)

f:\Python\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "f:\Python\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "f:\Python\lib\site-packages\sklearn\pipeline.py", line 378, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "f:\Python\lib\site-packages\sklearn\pipeline.py", line 336, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "f:\Python\lib\site-packages\joblib\memory.py", line 349, in __call__
    return se

   explained_variance:  0.4571
   r2:  0.4557
   MAE:  241.707
   MSE:  7350827.9633
   RMSE:  2711.241
{'estimator__n_estimators': 150, 'estimator__min_samples_split': 0.6300000363636363, 'estimator__max_features': 'log2', 'estimator__max_depth': 8, 'estimator__loss': 'squared_error', 'estimator__learning_rate': 0.04173469387755102, 'decomposer__n_components': 4}


In [ ]:
# loaded_model = joblib.load(filename)